In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법_with_table.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

194

In [5]:
document_list[46]

Document(metadata={'source': 'data/소득세법_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 

In [3]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

In [4]:
len(embedding.embed_query("소득세법"))

4096

In [6]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
index_name = "tax-index-table"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets

# 업로드한 벡터db를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 7.52 s
Wall time: 59.7 s


# 2. 답변 생성 전 retrieval 확인

In [7]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

In [30]:
retrieved_docs[1]

Document(id='1f01c88b-8700-45b9-b6e4-d9e21dffad91', metadata={'source': 'data/소득세법_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공

In [11]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [12]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [15]:
retrived_docs[1]

Document(id='1f01c88b-8700-45b9-b6e4-d9e21dffad91', metadata={'source': 'data/소득세법_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공

# 3. 답변 생성

In [17]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":"당신은 최고의 한국 소득세법 전문가입니다"},
        {
            "role":"user",
            "content": f"""- [context]를 참고해서 사용자의 질문에 10줄이내로 답변해 주세요
- [context] : {retrieved_doc}
- 질문 : {query}"""
        }
    ],
    temperature=0.2
)

In [20]:
print(response.choices[0].message.content)

연봉이 5천만원인 직장인의 소득세를 계산하기 위해 먼저 종합소득 과세표준을 확인해야 합니다. 

1. **종합소득 과세표준**: 5천만원
2. **세율 적용**:
   - 1,400만원 이하: 1,400만원 × 6% = 84만원
   - 1,400만원 초과 5,000만원 이하: 84만원 + (5,000만원 - 1,400만원) × 15% = 84만원 + 525만원 = 609만원

3. **근로소득세액공제**: 총급여액이 3천 300만원 초과 7천만원 이하이므로,
   - 공제액 = 74만원 - [(5,000만원 - 3,300만원) × 8/1000] = 74만원 - 13.6만원 = 60.4만원 (66만원으로 하한 적용)

4. **최종 소득세**: 609만원 - 66만원 = 543만원

따라서, 연봉이 5천만원인 직장인의 소득세는 약 543만원입니다.


In [21]:
from openai import OpenAI
import os
# upstage의 과금정책 : https://www.upstage.ai/pricing/api
client = OpenAI(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {"role":"system", "content":"당신은 최고의 한국 소득세법 전문가입니다"},
        {
            "role":"user",
            "content": f"""- [context]를 참고해서 사용자의 질문에 10줄이내로 답변해 주세요
- [context] : {retrieved_doc}
- 질문 : {query}"""
        }
    ],
    temperature=0.2
)

In [22]:
print(response.choices[0].message.content)

연봉 5천만원인 직장인의 소득세는 다음과 같이 계산됩니다(2024년 기준, 근로소득세액공제 적용):

1. **종합소득산출세액 계산**  
   - 1,400만원 이하: 1,400만원 × 6% = 84만원  
   - 1,400만원 초과 5,000만원 이하: (5,000만원 - 1,400만원) × 15% = 540만원  
   - **산출세액**: 84만원 + 540만원 = **624만원**

2. **근로소득세액공제 적용**  
   - 총급여액 3,300만원 초과 7,000만원 이하 구간:  
     `74만원 - [(5,000만원 - 3,300만원) × 8/1,000] = 74만원 - 12.8만원 = 61.2만원`  
   - 단, 최소 66만원 적용 → **66만원 공제**

3. **최종 세액**  
   - 624만원(산출세액) - 66만원(공제) = **558만원**  

※ 추가 공제(자녀세액공제 등) 적용 시 세액 감소 가능합니다.  

(계산: 종합소득산출세액 624만원 - 근로소득세액공제 66만원 = **558만원**)


In [24]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4.1-nano")

In [23]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [24]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [25]:
ai_message = llm.invoke(prompt)

In [26]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세를 계산하는 과정은 다음과 같습니다.

---

### **1. 종합소득과세표준 계산**
- **총급여액**: 5,000만원  
- **근로소득공제** (소득세법 제47조 기준):  
  - 500만원 이하: \(500 \times 70\% = 350\)만원  
  - 500만원 초과 1,500만원 이하: \(1,000 \times 40\% = 400\)만원  
  - 1,500만원 초과 4,500만원 이하: \(3,000 \times 15\% = 450\)만원  
  - 4,500만원 초과 5,000만원 이하: \(500 \times 5\% = 25\)만원  
  - **총 근로소득 공제액**: \(350 + 400 + 450 + 25 = 1,225\)만원  
- **종합소득과세표준**:  
  \(5,000\text{만원} - 1,225\text{만원} = 3,775\text{만원}\)

---

### **2. 종합소득산출세액 계산** (소득세법 제55조)
- **세율 적용**:  
  - 1,400만원 이하: \(1,400 \times 6\% = 84\)만원  
  - 1,400만원 초과 5,000만원 이하: \( (3,775 - 1,400) \times 15\% = 356.25\)만원  
  - **종합소득산출세액**: \(84 + 356.25 = 440.25\)만원

---

### **3. 근로소득세액공제 적용** (소득세법 제59조)
- **총급여액**: 5,000만원 (3,300만원 초과 7,000만원 이하 구간)  
- **공제액 계산**:  
  \[
  74\text{만원} - \left( \frac{5,000 - 3,300}{1,000} \times 8 \right) = 74 - 13.6 = 60.4\text{만원}
  \]  
  - 단, **66만원 미만 시 66만원 적용** → **66만원 공제**  
- **최종 세액**:  
  \(440.25\text{만원} - 66\text{만원} = 374.2

# 4. langchain 답변 생성
- ch9.07_LangChain과 vectorDatabase을 활용한 RAG구현(UpstageEmbedding) 참조

In [27]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model = "gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")

embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

# 2. # 업로드한 벡터db를 가져올 때
vectorstore = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])

In [28]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'연봉 5천만원인 직장인의 근로소득세액공제는 총급여액이 3,300만원 초과 7,000만원 이하에 해당하므로,  \n**66만원**이 적용됩니다(74만원 - [(5,000만원 - 3,300만원) × 8/1,000 = 66만원).  \n단, 이는 근로소득세액공제 금액이며, 실제 소득세는 종합소득산출세액에서 공제 후 계산됩니다.  \n\n(참고: 추가 공제 항목 및 과세표준 계산은 별도 필요)'